In [1]:
from asn1 import Encoder, Decoder, Numbers
from pygost.gost34112012256 import GOST34112012256
from random import randrange


# поместить подпись в asn1 файл
def sign_to_asn(a, b, p, q, r, s, x_q, y_q, x_p, y_p, file_name):
    # запись в asn1 (Приложение Г методичесого пособия)
    asn1 = Encoder()
    asn1.start()
    asn1.enter(Numbers.Sequence) # заголовок
    asn1.enter(Numbers.Set) # множество ключей, 1 задействован
    asn1.enter(Numbers.Sequence) # первый «ключ»
    asn1.write(b'\x80\x06\x07\x00', Numbers.OctetString) # идентификатор алгоритма (протокол подписи ГОСТ)
    asn1.enter(Numbers.Sequence) # значение открытого ключа
    asn1.write(int(x_q), Numbers.Integer) # x-координата точки Q
    asn1.write(int(y_q), Numbers.Integer) # y-координата точки Q
    asn1.leave()
    asn1.enter(Numbers.Sequence) # параметры криптосистемы
    asn1.enter(Numbers.Sequence) # параметры поля
    asn1.write(p, Numbers.Integer) # простое число p
    asn1.leave()
    asn1.enter(Numbers.Sequence) # параметры кривой
    asn1.write(a, Numbers.Integer) # коэффициент A уравнения кривой
    asn1.write(b, Numbers.Integer) # коэффициент B уравнения кривой
    asn1.leave()
    asn1.enter(Numbers.Sequence) # образующая группы точек кривой
    asn1.write(int(x_p), Numbers.Integer) # x-координата образующей точки P
    asn1.write(int(y_p), Numbers.Integer) # y-координата образующей точки P
    asn1.leave()
    asn1.write(q, Numbers.Integer) # порядок группы q
    asn1.leave()
    asn1.enter(Numbers.Sequence) # подпись сообщения
    asn1.write(int(r), Numbers.Integer) # число r
    asn1.write(int(s), Numbers.Integer) # число s
    asn1.leave()
    asn1.leave()
    asn1.leave()
    asn1.enter(Numbers.Sequence) # параметры файла, не используются
    asn1.leave()
    asn1.leave()
    with open(file_name + '.sign.asn1', 'wb') as output_file:
        output_file.write(asn1.output())


# достать подпись из asn1 файла
def sign_from_asn(file_name):
    with open(file_name + '.sign.asn1', "rb") as input_file:
        sign_data = input_file.read()
    asn1 = Decoder()
    asn1.start(sign_data)
    asn1.enter() # заголовок
    asn1.enter() # множество ключей, 1 задействован
    asn1.enter() # первый «ключ»
    value = asn1.read() # идентификатор алгоритма (протокол подписи ГОСТ)
    asn1.enter() # значение открытого ключа
    x_q = asn1.read()[1] # x-координата точки Q
    y_q = asn1.read()[1] # y-координата точки Q
    asn1.leave()
    asn1.enter() # параметры криптосистемы
    asn1.enter() # параметры поля
    p = asn1.read()[1] # простое число p
    asn1.leave() 
    asn1.enter() # параметры кривой
    a = asn1.read()[1] # коэффициент A уравнения кривой
    b = asn1.read()[1] # коэффициент B уравнения кривой
    asn1.leave()
    asn1.enter() # образующая группы точек кривой
    x_p = asn1.read()[1] # x-координата образующей точки P
    y_p = asn1.read()[1] # y-координата образующей точки P
    asn1.leave()
    q = asn1.read()[1] # порядок группы q
    asn1.leave()
    asn1.enter() # подпись сообщения
    r = asn1.read()[1] # число r
    s = asn1.read()[1] # число s
    asn1.leave()
    asn1.leave()
    asn1.leave()
    asn1.enter() # параметры файла, не используются
    asn1.leave()
    asn1.leave()
    return a, b, p, q, r, s, x_q, y_q, x_p, y_p


# формирование подписи ГОСТ Р 34.10-2018
def sign_gost34102018(a, b, p, q, d, x_p, y_p, file_name):
    # читаем файл с содержимым, которое подпишем
    with open(file_name, "rb") as input_file:
        data = input_file.read()
    # задаем эллиптическую кривую
    K = GF(p)
    E = EllipticCurve(K, [a, b])
    P = E(x_p, y_p)
    # 1
    h = GOST34112012256(data).digest()
    # 2
    alpha = int.from_bytes(h, byteorder='big')
    e = mod(alpha, q)
    if e == 0:
        e = 1
    while True:
        # 3
        k = randrange(1, q)
        # 4
        C = k * P
        r = mod(C[0], q)
        if r == 0:
            continue
        # 5
        s = mod(r * d + k * e, q)
        if s == 0:
            continue
        break
    Q = d * P
    sign_to_asn(a, b, p, q, r, s, Q[0], Q[1], P[0], P[1], file_name)
    print('Файл ' + file_name + ' подписан. Файл с подписью: ' + file_name + '.sign.asn1')


# проверка подписи ГОСТ Р 34.10-2018
def check_sign_gost34102018(file_name):
    a, b, p, q, r, s, x_q, y_q, x_p, y_p = sign_from_asn(file_name)
    # читаем файл с содержимым, подпись которого проверяем
    with open(file_name, "rb") as input_file:
        data = input_file.read()
    # задаем эллиптическую кривую
    K = GF(p)
    E = EllipticCurve(K, [a, b])
    P = E(x_p, y_p)
    Q = E(x_q, y_q)
    # 1
    if not ((0 < r and r < q) and (0 < s and s < q)):
        print('Подпись файла ' + file_name + ' неверна!')
        return
    # 2
    h = GOST34112012256(data).digest()
    # 3
    alpha = int.from_bytes(h, byteorder='big')
    e = mod(alpha, q)
    if e == 0:
        e = 1
    # 4.
    v = inverse_mod(e, q)
    # 5
    z_1 = int(mod(s * v, q))
    z_2 = int(mod(-r * v, q))
    # 6
    C = z_1 * P + z_2 * Q
    R = mod(C[0], q)
    if R == r:
        print('Подпись файла ' + file_name + ' верна!')
    else:
        print('Подпись файла ' + file_name + ' неверна!')


print('Схема подписи ГОСТ Р 34.10-2018')
file_name = 'C:\\test\\image.png'
while True:
    command = int(input('Введите команду:\n1 - подписать файл\n2 - проверить подпись файла\n3 - выйти\n'))
    if command == 1:
        # Параметры криптосистемы заданы статически, в реальных условиях необходим генератор параметров
        a = -1
        b = 53956679838042162451108292176931772631109916272820066466458395232513766926866
        p = 57896044625259982827082014024491516445703215213774687456785671200359045162371
        r = 28948022312629991413541007012245758222850495633896873081323396140811733708403
        x = 12933162268009944794066590054824622037560826430730236852169234350278155715869
        y = 18786030474197088418858017573086015439132081546303111294023901101650919011383
        q = r
        d = 1732504998467916685624752132319238521322532122193426470314523640303299087831
        sign_gost34102018(a, b, p, q, d, x, y, file_name)
    elif command == 2:
        check_sign_gost34102018(file_name)
    elif command == 3:
        break
    else:
        print('Неизвестная команда!')

Схема подписи ГОСТ Р 34.10-2018
Введите команду:
1 - подписать файл
2 - проверить подпись файла
3 - выйти
3
